# glTF (Graphics Language Transmission Format)

Material Specification is described in https://registry.khronos.org/glTF/specs/2.0/glTF-2.0.html#appendix-b-brdf-implementation



In [ ]:
from glTF import *


brdf = glTF_brdf()
base_color, alpha, metallic, ior = brdf.bsdf_params.values()
ior = sp.Symbol(r'\mathrm{ior}')

# ior = 1.5 # default without KHR_materials_ior

display(sp.Eq(sp.Symbol("gltf"), brdf.bsdf))

# Material Structure

    material = mix(dielectric_brdf, metal_brdf, metallic)
         = (1.0 - metallic) * dielectric_brdf + metallic * metal_brdf


## Metals

metal_brdf =
  conductor_fresnel(
    f0 = baseColor,
    bsdf = specular_brdf(
      α = roughness ^ 2))

## Dielectrics

    dielectric_brdf =
        fresnel_mix(
            ior = 1.5,
            base = diffuse_brdf(
            color = baseColor),
            layer = specular_brdf(
            α = roughness ^ 2))

# Diffuse Component

In [ ]:
diffuse = diffuse_component(C_SYM)
display(diffuse)
display(sp.Eq(sp.MatrixSymbol("LambertianBRDF", 3, 1), diffuse))

# Specular Component

In [ ]:
display(sp.Eq(H_SYM, half_vector(V, N, L)))
display(sp.Eq(H_SYM, half_vector(V_SYM, N_SYM, L_SYM)))

D_sym, VX_sym = sp.symbols("D V_ggx")

display(sp.Eq(D_sym, specular_D_GGX(N_SYM, H_SYM, alpha)))

display(sp.Eq(VX_sym, specular_V_GGX(V_SYM, N_SYM, L_SYM, alpha)))

display(sp.Eq(sp.Symbol("MicrofacetBRDF"), specular_component(VX_sym, D_sym)))

microfacet_brdf = specular_component(specular_V_GGX(V_SYM, N_SYM, L_SYM, alpha), specular_D_GGX(N_SYM, H_SYM, alpha)).simplify()
display(sp.Eq(sp.Symbol("MicrofacetBRDF"), microfacet_brdf))


# Dielectric Component

In [ ]:
display(M1_SYM * microfacet_brdf)
dielectric_brdf = fresnel_mix(V_SYM, H_SYM, ior, diffuse_component(M1_SYM), M1_SYM * microfacet_brdf)
display(sp.Eq(sp.MatrixSymbol("dielectric", 3, 1), dielectric_brdf))

# Metal Component

In [ ]:
metal_brdf = conductor_fresnel(
    V, H, base_color, microfacet_brdf
)
display(sp.Eq(sp.MatrixSymbol("metal", 3, 1), metal_brdf))

# full glTF 

In [ ]:
gltf_sym = sp.MatrixSymbol("gltf",3, 1)

gltf_brdf = mix(dielectric_brdf, metal_brdf, metallic)
display(sp.Eq(gltf_sym, gltf_brdf))

guarded_gltf_brdf = gltf(V, N, L, base_color, metallic, alpha)
display(sp.Eq(sp.Symbol("gltf"), guarded_gltf_brdf))
display(sp.Eq(gltf_sym, guarded_gltf_brdf))


In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

from bsdf import *
from glTF import *


theta_v = 15 / 90.0 * np.pi / 2
# phi_v = 
rho_val = np.array([1, 0.5, 0.5])
roughness_val = 0.2
metallic_val = 0.0
ior_val = 1.5

N_val = np.array([0, 0, 1], dtype=np.float32)
V_val = np.array([np.sin(theta_v), 0, np.cos(theta_v)], dtype=np.float32)
brdf = glTF_brdf()
brdf_np = brdf.get_np()

print(
    "Integral:",
    integrate_spherical_function(lambda l: brdf_np(
        V_val, N_val, l, rho_val, roughness_val**2, metallic_val, 1.5) * np.abs(np_dot(l, N_val)), 10000),
)
plot_brdf(
    "glTF",
    lambda v, n, l: brdf_np(
        v, n, l, rho_val, roughness_val**2, metallic_val, ior_val),
    V_val,
    normalize=False,
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from bsdf import *
from glTF import *
import merl


def linear_to_srgb(c):
    return np.where(
        c <= 0.0031308,
        12.92 * c,
        (1.0 + 0.055) * np.power(c, 1.0 / 2.4) - 0.055
    )

# merl_data = merl.read_merl_brdf(os.path.join(dir, f"{material}.binary"))


theta_h, _, theta_d, _ = merl.generate_dense_half_diffs(0)
theta_h, theta_d = np.meshgrid(theta_h, theta_d, indexing='ij')
theta_o, phi_o, theta_i, phi_i = merl.half_diff_to_std_coords(
    theta_h, 0, theta_d, np.pi / 2
)
n = np.array([0, 0, 1])
v = np.stack((np.sin(theta_o) * np.cos(phi_o), np.sin(theta_o)
             * np.sin(phi_o), np.cos(theta_o)), axis=-1)
l = np.stack((np.sin(theta_i) * np.cos(phi_i), np.sin(theta_i)
             * np.sin(phi_i), np.cos(theta_i)), axis=-1)


brdf = glTF_brdf()
brdf_np = brdf.get_np()
rho_val = np.array([1, 0.2, 0.2])
roughness_val = 0.01
metallic_val = 0

model_output = brdf_np(v, n, l, rho_val, roughness_val**2, metallic_val, 1.5)
model_output = np.swapaxes(model_output, 0, -1).reshape(90, 90, 3)


fig, ax = plt.subplots(1, 1)
ax.set_xticks([])
ax.set_xticks([], minor=True)
ax.set_yticks([])
ax.set_yticks([], minor=True)
ax.imshow(np.clip(linear_to_srgb(model_output), 0, 1), origin='lower')
plt.show()